In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
pd.set_option('display.max_columns', 121)

Data yang digunakan adalah data yang telah dikoreksi baseline

In [2]:
A_csv_file = glob.glob("../data_baseline/_A*.csv") # Mencari semua berkas CSV yang dimulai dengan 'A' dalam direktori 'data'
B_csv_file = glob.glob("../data_baseline/_B*.csv") # Mencari semua berkas CSV yang dimulai dengan 'B' dalam direktori 'data'

In [3]:
def get_MaxGradient(dataframe, column):
    array = dataframe[column].to_numpy()
    return max(np.gradient(array, edge_order=2))

In [4]:
def extract_FeaturesCubic_3W(CSV_files, label, return_dataset=False):
    """extract max, mean, median, and gradients with two windows method from given CSV files

    Args:
        CSV_files (list): csv files
        label (str): label
        return_dataset (bool, optional): wheter get formated dataset. Defaults to False.

    Returns:
        DataFrame: if return_dataset is True
        array: if return_dataset is False
    """
    maxi1 = np.empty((0, 10))
    maxi2 = np.empty((0, 10))
    maxi3 = np.empty((0, 10))
    mean1 = np.empty((0, 10))
    mean2 = np.empty((0, 10))
    mean3 = np.empty((0, 10))
    median1 = np.empty((0, 10))
    median2 = np.empty((0, 10))
    median3 = np.empty((0, 10))
    grad1 = np.empty((0, 10))
    grad2 = np.empty((0, 10))
    grad3 = np.empty((0, 10))
    for csv_file in CSV_files:
        df = pd.read_csv(csv_file)

        first_window = df[(df['time(s)'] >= 10) & (df['time(s)'] <= 15)]
        second_window = df[(df['time(s)'] > 15) & (df['time(s)'] <= 20)]
        third_window = df[(df['time(s)'] > 20) & (df['time(s)'] <= 30)]

        df.drop(columns=["time(s)","Temp","Humid"], inplace=True)

        _max_list1 = []
        _max_list2 = []
        _max_list3 = []
        _mean_list1 = []
        _mean_list2 = []
        _mean_list3 = []
        _median_list1 = []
        _median_list2 = []
        _median_list3 = []
        _grad_list1 = []
        _grad_list2 = []
        _grad_list3 = []

        for cols, _ in df.iteritems():        
            _max_list1.append(first_window[cols].max())
            _max_list2.append(second_window[cols].max())
            _max_list3.append(third_window[cols].max())

            _mean_list1.append(first_window[cols].mean())
            _mean_list2.append(second_window[cols].mean())
            _mean_list3.append(third_window[cols].mean())

            _median_list1.append(first_window[cols].median())
            _median_list2.append(second_window[cols].median())
            _median_list3.append(third_window[cols].median())

            _grad_list1.append(get_MaxGradient(first_window, cols))
            _grad_list2.append(get_MaxGradient(second_window, cols))
            _grad_list3.append(get_MaxGradient(third_window, cols))

        maxi1 = np.vstack((maxi1, np.array(_max_list1).T))
        maxi2 = np.vstack((maxi2, np.array(_max_list2).T))
        maxi3 = np.vstack((maxi3, np.array(_max_list3).T))
        
        mean1 = np.vstack((mean1, np.array(_mean_list1).T))
        mean2 = np.vstack((mean2, np.array(_mean_list2).T))
        mean3 = np.vstack((mean3, np.array(_mean_list3).T))
        
        median1 = np.vstack((median1, np.array(_median_list1).T))
        median2 = np.vstack((median2, np.array(_median_list2).T))
        median3 = np.vstack((median3, np.array(_median_list3).T))
        
        grad1 = np.vstack((grad1, np.array(_grad_list1).T))
        grad2 = np.vstack((grad2, np.array(_grad_list2).T))
        grad3 = np.vstack((grad3, np.array(_grad_list3).T))
    
    if return_dataset:
        mean_df_1 = pd.DataFrame(mean1, columns=[f"mu1{i+1}" for i in range(10)])
        mean_df_2 = pd.DataFrame(mean2, columns=[f"mu2{i+1}" for i in range(10)])
        mean_df_3 = pd.DataFrame(mean3, columns=[f"mu3{i+1}" for i in range(10)])
        
        max_df_1 = pd.DataFrame(maxi1, columns=[f"max1{i+1}" for i in range(10)])
        max_df_2 = pd.DataFrame(maxi2, columns=[f"max2{i+1}" for i in range(10)])
        max_df_3 = pd.DataFrame(maxi3, columns=[f"max3{i+1}" for i in range(10)])
        
        median_df_1 = pd.DataFrame(median1, columns=[f"med1{i+1}" for i in range(10)])
        median_df_2 = pd.DataFrame(median2, columns=[f"med2{i+1}" for i in range(10)])
        median_df_3 = pd.DataFrame(median3, columns=[f"med3{i+1}" for i in range(10)])
        
        grad_df_1 = pd.DataFrame(grad1, columns=[f"grad1{i+1}" for i in range(10)])
        grad_df_2 = pd.DataFrame(grad2, columns=[f"grad2{i+1}" for i in range(10)])
        grad_df_3 = pd.DataFrame(grad3, columns=[f"grad3{i+1}" for i in range(10)])
        
        dataset = pd.concat([mean_df_1, max_df_1, median_df_1, grad_df_1,
                             mean_df_2, max_df_2, median_df_2, grad_df_2,
                             mean_df_3, max_df_3, median_df_3, grad_df_3], axis=1)
        dataset["label"] = label
        
        return dataset
        
    else:        
        all_arrays = [maxi1, maxi2, maxi3, mean1, mean2, mean3, median1, median2, median3, grad1, grad2, grad3]
        all_arrays = np.transpose(all_arrays, (1, 2, 0))
        return all_arrays

In [5]:
A_array = extract_FeaturesCubic_3W(A_csv_file, label="A")
B_array = extract_FeaturesCubic_3W(B_csv_file, label="B")
print(A_array.shape)
print(B_array.shape)

(35, 10, 12)
(35, 10, 12)


In [6]:
A_array_file_filename = 'A_array_3W.npy'
B_array_file_filename = 'B_array_3W.npy'

np.save(f"../Exp/{A_array_file_filename}", A_array)
np.save(f"../Exp/{B_array_file_filename}", B_array)

In [7]:
A_tensor =  tf.convert_to_tensor(A_array)
B_tensor =  tf.convert_to_tensor(B_array)
print(A_tensor.shape)
print(B_tensor.shape)

(35, 10, 12)
(35, 10, 12)


In [8]:
A_tensor_file_filename = 'A_tensor_3W.tf'
B_tensor_file_filename = 'B_tensor_3W.tf'

tf.io.write_file(f"../Exp/{A_tensor_file_filename}", tf.io.serialize_tensor(A_tensor))
tf.io.write_file(f"../Exp/{B_tensor_file_filename}", tf.io.serialize_tensor(B_tensor))

In [9]:
# how to load
# tensor_bytes = tf.io.read_file(file_path)

# tensor_tf = tf.io.parse_tensor(tensor_bytes, out_type=tf.int32)

###  Get Dataset

In [10]:
A_dataset = extract_FeaturesCubic_3W(A_csv_file, label="A", return_dataset=True)
B_dataset = extract_FeaturesCubic_3W(B_csv_file, label="B", return_dataset=True)

In [11]:
full_dataset = pd.concat([A_dataset, B_dataset], axis=0)
full_dataset.reset_index(drop=True, inplace=True)

In [12]:
full_dataset.head()

,mu11,mu12,mu13,mu14,mu15,mu16,mu17,mu18,mu19,mu110,max11,max12,max13,max14,max15,max16,max17,max18,max19,max110,med11,med12,med13,med14,med15,med16,med17,med18,med19,med110,grad11,grad12,grad13,grad14,grad15,grad16,grad17,grad18,grad19,grad110,mu21,mu22,mu23,mu24,mu25,mu26,mu27,mu28,mu29,mu210,max21,max22,max23,max24,max25,max26,max27,max28,max29,max210,med21,med22,med23,med24,med25,med26,med27,med28,med29,med210,grad21,grad22,grad23,grad24,grad25,grad26,grad27,grad28,grad29,grad210,mu31,mu32,mu33,mu34,mu35,mu36,mu37,mu38,mu39,mu310,max31,max32,max33,max34,max35,max36,max37,max38,max39,max310,med31,med32,med33,med34,med35,med36,med37,med38,med39,med310,grad31,grad32,grad33,grad34,grad35,grad36,grad37,grad38,grad39,grad310,label
0,131.579338,8.063088,-1.047255,525.389387,251.257108,190.941765,4.633137,800.540172,512.948284,964.983358,457.98375,27.46750,-0.28500,1454.92125,758.1125,533.78000,11.91500,2195.48625,1416.36250,2089.96375,67.98375,4.34250,-1.16000,309.29625,158.2375,136.28000,3.91500,431.98625,341.36250,968.96375,18.9375,1.3125,0.0625,63.8125,27.0625,18.5000,0.4375,107.3750,56.1875,105.3125,869.12625,60.70500,0.28500,1637.31375,961.2100,639.88000,16.25250,2373.68375,1687.47500,2234.95625,1158.35875,93.34250,1.46500,1683.54625,1013.8625,666.03000,19.16500,2415.36125,1789.61250,2276.46375,898.23375,60.59250,0.34000,1665.48375,995.6125,655.28000,16.66500,2399.11125,1722.61250,2256.27625,19.00,1.4375,0.1875,17.1250,17.1875,9.5000,0.3125,18.8125,21.2500,13.0625,1280.08500,140.87625,2.85125,1558.58875,772.5625,521.21625,20.04375,2347.25375,1756.88500,2182.40625,1310.98375,170.34250,3.71500,1678.92125,990.3625,648.15500,20.54000,2413.98625,1795.73750,2275.21375,1292.48375,145.34250,2.96500,1569.60875,776.3625,521.21750,20.16500,2355.36125,1766.30000,2193.83875,7.1250,1.3125,0.1875,-1.000,-2.7500,-1.6875,0.1875,-0.3125,0.8750,-0.3750,A
1,59.562721,3.265270,-0.124020,311.589755,105.164608,86.858039,1.998333,428.542819,266.773064,593.225735,208.88625,10.84125,0.30000,999.07750,327.3950,253.26000,5.41500,1525.12125,825.36375,1590.26250,31.88625,1.84125,-0.20000,161.32750,67.7700,60.88500,1.66500,169.99625,155.36375,397.76250,9.0000,0.5000,0.0625,38.5000,11.1875,8.3750,0.3125,63.8125,29.3125,65.5625,453.48375,25.06375,0.82750,1322.73500,493.0350,350.47000,7.65750,2085.57375,1181.05875,1864.34000,675.13625,39.59125,1.42500,1448.45250,569.0200,393.51000,9.29000,2298.62125,1379.48875,1978.26250,457.57375,24.90375,0.80000,1365.20250,514.6450,363.38500,7.79000,2157.55875,1216.61375,1897.82500,10.00,0.6250,0.1875,24.7500,10.0625,6.0625,0.1875,45.1875,21.5625,20.9375,903.54500,63.82125,2.19000,1390.89500,488.8850,350.01250,10.34125,2296.51000,1441.98625,1934.93875,1000.63625,80.84125,2.80000,1454.20250,569.6450,393.88500,10.79000,2325.49625,1459.61375,1986.51250,935.57375,65.52875,2.30000,1404.64000,496.2700,354.38500,10.41500,2307.18375,1447.67625,1949.38750,7.5000,0.6250,0.1875,1.125,0.3125,0.1875,0.1875,2.3125,4.0000,1.0625,A
2,78.161348,4.082672,0.090858,332.101740,125.539314,104.979804,2.434044,509.280025,294.737132,667.627255,264.79125,13.90375,0.32125,1051.00125,396.1300,308.72000,6.19875,1654.34375,895.28125,1659.49000,43.91625,2.02875,0.07125,167.50125,80.2550,73.47000,2.07375,227.59375,173.28125,519.36500,11.0625,0.6250,0.0625,41.1250,13.5625,10.1250,0.3750,68.3750,32.1875,71.8750,555.55875,32.39125,0.94125,1342.97375,597.1900,421.89000,8.94125,2057.51375,1229.20875,1891.21000,806.41625,51.40375,1.69625,1450.00125,687.0050,467.47000,10.82375,2197.59375,1401.40625,1985.61500,563.79125,32.15375,0.94625,1381.68875,624.1300,437.72000,9.13625,2109.78125,1265.03125,1919.80250,12.00,0.9375,0.1250,23.5625,12.4375,7.4375,0.1875,35.5000,21.5625,18.0625,1025.03750,83.51875,2.78875,1392.31000,598.5700,416.45000,12.24250,2180.53250,1441.64875,1947.97375,1100.79125,106.40375,3.57125,1453.50125,687.7550,467.47000,12.82375,2209.21875,1457.03125,1992.49000,1059.66625,85.84125,2.82125,1403.87625,607.4425,420.97000,12.44875,2190.34375,1445.4687

In [13]:
full_dataset.tail()

,mu11,mu12,mu13,mu14,mu15,mu16,mu17,mu18,mu19,mu110,max11,max12,max13,max14,max15,max16,max17,max18,max19,max110,med11,med12,med13,med14,med15,med16,med17,med18,med19,med110,grad11,grad12,grad13,grad14,grad15,grad16,grad17,grad18,grad19,grad110,mu21,mu22,mu23,mu24,mu25,mu26,mu27,mu28,mu29,mu210,max21,max22,max23,max24,max25,max26,max27,max28,max29,max210,med21,med22,med23,med24,med25,med26,med27,med28,med29,med210,grad21,grad22,grad23,grad24,grad25,grad26,grad27,grad28,grad29,grad210,mu31,mu32,mu33,mu34,mu35,mu36,mu37,mu38,mu39,mu310,max31,max32,max33,max34,max35,max36,max37,max38,max39,max310,med31,med32,med33,med34,med35,med36,med37,med38,med39,med310,grad31,grad32,grad33,grad34,grad35,grad36,grad37,grad38,grad39,grad310,label
65,93.074314,4.935123,0.123358,372.255662,146.233407,129.277966,4.167770,596.016912,285.548505,789.512426,318.41500,16.87875,0.85375,1079.35125,468.09125,364.11375,10.61875,1816.86250,815.16125,1788.19625,52.29000,2.62875,-0.02125,226.85125,87.71625,96.23875,3.61875,295.23750,195.91125,749.94625,13.2500,0.7500,0.1875,42.6875,16.4375,11.9375,0.4375,76.8750,29.1250,87.0625,641.14500,39.17125,1.68125,1287.77625,682.41625,471.98125,14.64875,2119.58000,1054.55375,1969.02375,908.04000,62.25375,2.47875,1362.47625,775.59125,514.48875,17.61875,2218.73750,1173.78625,2044.82125,653.29000,38.75375,1.72875,1315.03875,711.02875,486.80125,14.86875,2158.80000,1080.78625,1990.94625,13.5000,1.0000,0.0625,17.1875,13.7500,7.6250,0.2500,27.8750,15.8125,13.6875,1121.27125,104.09625,3.30375,1318.08125,705.84750,473.04125,20.08625,2206.41875,1199.65875,2029.38875,1191.29000,135.75375,3.85375,1364.85125,777.71625,514.98875,21.11875,2226.86250,1211.66125,2054.19625,1155.72750,106.62875,3.35375,1328.47625,717.84125,479.30125,20.49375,2214.11250,1202.47375,2039.88375,8.3750,1.0000,0.1875,0.5625,0.6875,0.1250,0.1875,0.9375,2.1875,1.0625,B
66,79.638309,3.518701,1.271887,321.225098,126.929093,111.659314,5.579608,524.415025,266.443333,714.003627,271.61625,13.76625,3.30375,999.98000,408.19625,319.75000,13.43500,1690.97875,782.11000,1717.18500,44.11625,1.39125,1.05375,169.10500,75.44625,79.00000,4.68500,240.47875,174.11000,606.81000,11.4375,0.8750,0.1250,39.0625,14.5625,10.5000,0.4375,69.8750,27.3125,73.7500,566.58125,33.39875,4.41125,1260.19500,616.26625,430.09000,18.68000,2078.83875,1052.41250,1929.05750,816.36625,53.26625,4.92875,1350.73000,703.19625,471.25000,22.18500,2207.10375,1187.86000,2013.18500,576.24125,33.20375,4.55375,1295.41750,645.69625,446.31250,19.06000,2130.04125,1082.42250,1956.18500,12.2500,0.8750,0.1250,21.7500,13.1875,7.6250,0.3125,35.1250,17.7500,16.7500,1022.82000,86.22125,4.49250,1280.24875,601.14625,411.79125,23.89125,2180.81750,1206.41500,1966.52500,1089.74125,108.76625,4.92875,1351.98000,703.44625,471.12500,24.68500,2215.10375,1223.73500,2017.56000,1057.11625,88.76625,4.49125,1292.60500,610.94625,416.62500,23.93500,2190.97875,1211.11000,1979.62250,7.9375,0.8750,0.0625,0.2500,0.2500,-0.0625,0.1250,1.0625,2.2500,0.6250,B
67,96.897647,5.575735,0.414485,395.889044,153.717010,134.024167,4.231716,622.141740,337.214265,831.861324,339.53000,18.86250,1.32625,1170.77875,504.19250,389.48250,11.19250,1915.04125,955.05250,1894.42750,51.65500,2.98750,0.32625,223.90375,89.31750,95.35750,3.69250,293.66625,230.42750,767.80250,14.6250,0.8750,0.1875,47.0625,18.3750,12.9375,0.4375,83.4375,34.4375,89.8750,693.88250,44.09750,2.39375,1399.68375,746.21000,512.27750,15.76000,2218.47125,1216.98250,2072.73750,980.65500,70.23750,3.32625,1479.65375,847.19250,557.35750,18.94250,2312.29125,1340.67750,2140.67750,709.03000,43.67500,2.45125,1429.90375,779.25500,529.92000,16.00500,2258.10375,1246.92750,2095.61500,14.9375,1.1875,0.1250,19.1250,15.5625,8.7500,0.5625,28.5625,18.1250,14.1875,1178.25375,115.48000,4.71125,1416.29375,740.62250,492.21750,21.30250,2285.74500,1350.48250,2099.84375,1232.90500,147.61250,5.57625,1480.77875,847.69250,557.23250,22.06750,2316.54125,1367.67750,2143.55250,1214.09250,118.73750,4.82625,1427.77875,752.44250,497.3

#### Save to csv file

In [14]:
full_dataset.to_csv("../dataset/data_2c40d3w.csv", index=False)